In [1]:
import json
from pathlib import Path
import sys
from functools import partial
from tqdm import tqdm_notebook as tqdm
sys.path.append("../")

import html2text

h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_pictures = True
h.ignore_images = True
h.ignore_emphasis = True

In [2]:
from src.data.utils import read_jsonl, extract_text_by_headers_html, dedup_results, html2text_parser, html2text_parser_traverse
import torch
# from src.models.ir.crossencoder import CrossEncoderIR
from src.models.ir import CrossEncoderIR, BiEncoderIR, TFIDF_IR, EnsembleIR
from src.models.llm.llama import LlamaLLM
from src.models.baseline_pipeline import BaselinePipeline
# from sentence_transformers import CrossEncoder
# from src.models.ir.tfidf import TFIDF_IR


# class CrossEncoderIR:
#     def __init__(self, model_name="cross-encoder/ms-marco-MiniLM-L-6-v2", max_sentences=10, max_len=128, score_threshold=0, device="cuda"):
#         self.model_name = model_name
#         self.max_sentences = max_sentences
#         self.max_len = max_len
#         self.model = CrossEncoder(model_name, max_length=max_len, device=device)
#         self.device = device
#         self.score_threshold = score_threshold
#         self.batch_size = 128


#     def get_top_sentences(self, query, candidates):
#         score_tuples = []
#         i = 0
#         while i < len(candidates):
#             batch_candidates = candidates[i:i+self.batch_size]
#             i += self.batch_size
#             input_tuples = [(query, candidate) for candidate in batch_candidates]
#             with torch.no_grad():
#                 scores = self.model.predict(input_tuples).tolist()
#             score_tuples.extend(list(zip(batch_candidates, scores)))
#         sorted_score_tuples = sorted(score_tuples, key=lambda x: -x[1])
#         sorted_score_tuples = [score_tuple for score_tuple in sorted_score_tuples if score_tuple[1] >= self.score_threshold][:self.max_sentences]
#         return sorted_score_tuples


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
json_objs = read_jsonl("../data/raw/task1/crag_task_1_dev_v3_release.jsonl", 500)
json_objs_split1 = [json_obj for json_obj in json_objs if json_obj["split"] == 1]
json_objs = [json_obj for json_obj in json_objs if json_obj["split"] == 0]
# preprocessing_fn = partial(extract_text_by_headers_html, version="v2", split_sentences=False)
# preprocessing_fn = html2text_parser
# preprocessing_fn = partial(html2text_parser, version="v5")
preprocessing_fn = html2text_parser_traverse

# ir_model = CrossEncoderIR(max_sentences=20, score_threshold=-1)
ir_models = [
    TFIDF_IR(max_sentences=None).load_model(),
    BiEncoderIR(max_sentences=None),
    CrossEncoderIR(max_sentences=None, score_threshold=float("-inf")),
]
ir_model = EnsembleIR(ir_models, max_sentences=20)
# ir_model = EnsembleIR(max_sentences=10, )
# ir_model = TFIDF_IR(max_sentences=20)
# ir_model.load_model()
pipeline = BaselinePipeline(preprocessing_fn, ir_model)

/home/mitchell/software/miniconda3/envs/meta_rag/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
print(len(json_objs_split1))
print(len(json_objs))

259
241


In [5]:
i = 0
answer, alt_answer, query, search_results, query_time = json_objs[i]["answer"], json_objs[i]["alt_ans"], json_objs[i]["query"], json_objs[i]["search_results"], json_objs[i]["query_time"]
deduped_search_results = dedup_results(search_results)
candidates = []
print(len(search_results), len(deduped_search_results))
for i, search_result in enumerate(deduped_search_results):
    with open(f"htmls/{i}.html", "w") as f:
        print(search_result["page_result"], file=f)
    print(search_result["page_url"])
    with open(f"segments/{i}.txt", "w") as f:
        for segment in preprocessing_fn(search_result):
            candidates.append(segment)
            print(segment, file=f)
            print("="*100, file=f)

top_segments = ir_model.get_top_sentences(query, candidates)
segments_text = [f"<DOC> {segment[1]}\n{segment[0]}\n</DOC>" for segment in top_segments]
segments_text = '\n\n'.join(segments_text).strip()
text_representation = f"<DOCS>\n{segments_text}\n</DOCS>"
print(query, query_time)
print()
print(answer)
print(alt_answer)
print(text_representation)

5 4
https://www.basketball-reference.com/players/n/nashst01.html
https://en.wikipedia.org/wiki/Steve_Nash
https://www.nba.com/stats/player/959/career
https://www.britannica.com/biography/Steve-Nash
how many 3-point attempts did steve nash average per game in seasons he made the 50-40-90 club? 03/10/2024, 23:19:21 PT

4 3-points attempts per game
[]
<DOCS>
<DOC> 3.3333333333333335
Steve Nash - Wikipedia
## Player profile
Nash is known for his dribbling, passing, and shooting abilities.
Nash was praised for his playmaking, ball-handling skills and shooting. He led
the league in assists for five years, averaging 11.5 assists per game in
2004–05, 10.5 in 2005–06, 11.6 in 2006–07, 11.0 in 2009–10 and 11.4 in
2010–11[16] and won the NBA Skills Challenge in 2005 and 2010.[12] As of the
end of 2012–13 season, he had a 90.4% career free throw shooting average
(formerly the best in NBA history, as of 2013[update] second to Stephen
Curry)[107][108] and a 42.8% career three-point shooting average 

In [8]:
# for search_result in search_results:
#     print(h.handle(search_result["page_result"]))

In [9]:
# def evaluate_pipeline(json_objs, pipeline):
    # results = []
    # for json_obj in json_objs:
    #     query = json_obj["query"]
    #     search_results = json_obj["search_results"]
    #     true_answer = json_obj["answer"]
    #     pred_answer = pipeline.generate_answer(query, search_results)
    #     results.append(
    #         {true_answer: pred_answer}
    #     )
    # return results

In [10]:
results = []
for json_obj in tqdm(json_objs):

    answer, query, search_results, query_time = json_obj["answer"], json_obj["query"], json_obj["search_results"], json_obj["query_time"]
    deduped_search_results = dedup_results(search_results)
    candidates = []
    for i, search_result in enumerate(deduped_search_results):
        with open(f"htmls/{i}.html", "w") as f:
            print(search_result["page_result"], file=f)
        for segment in preprocessing_fn(search_result):
            candidates.append(segment)

    top_segments = ir_model.get_top_sentences(query, candidates)
    segments_text = [f"<DOC> {segment[1]}\n{segment[0]}\n</DOC>" for segment in top_segments]
    segments_text = '\n\n'.join(segments_text).strip()
    text_representation = f"<DOCS>\n{segments_text}\n</DOCS>"

    query = json_obj["query"]
    search_results = json_obj["search_results"]
    true_answer = json_obj["answer"]
    pred_answer = pipeline.generate_answer(query, search_results, query_time)
    results.append(
        {
            "query": query,
            "query_time": query_time,
            "ground_truth": true_answer,
            "prediction": pred_answer,
            "partial_prompt": text_representation
        }
    )
# for evaluate_pipeline(json_objs, pipeline)

/tmp/ipykernel_228948/1519674984.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for json_obj in tqdm(json_objs):


  0%|          | 0/241 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/mitchell/software/miniconda3/envs/meta_rag/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Se

In [11]:
for i, result in enumerate(results):
    print(i)
    print(result["query"])
    print(result["ground_truth"])
    print(result["prediction"])
    print("-"*100)

0
how many 3-point attempts did steve nash average per game in seasons he made the 50-40-90 club?
4 3-points attempts per game
Insufficient information
----------------------------------------------------------------------------------------------------
1
what company in the dow jones is the best performer today?
salesforce
According to the provided information, Walmart (WMT) was one of the biggest gainers in the Dow Jones Industrial Average, up 1.6% on Tuesday.
----------------------------------------------------------------------------------------------------
2
in 2004, which animated film was recognized with the best animated feature film oscar?
finding nemo
In 2004, the animated film that was recognized with the Best Animated Feature Oscar was "The Incredibles".
----------------------------------------------------------------------------------------------------
3
which company in the s&p 500 index has the highest percentage of green energy usage?
the company with the highest percent

In [ ]:
with open("results.json", "w") as f:
    json.dump(results, f)